# **Step 1: Creating Tokens**

In [32]:
with open('/content/J. K. Rowling - Harry Potter 1 - Sorcerer\'s Stone.txt', 'r' , encoding="utf-8") as file:
    raw_text = file.read()

print("Total No of Characters" , len(raw_text))
print(raw_text[:50])

Total No of Characters 439742
Harry Potter and the Sorcerer's Stone


CHAPTER ON


In [33]:
import re

text = "Hello. how are you doing! as of now?"
result = re.split(r'(\s)', text)
print(result)

['Hello.', ' ', 'how', ' ', 'are', ' ', 'you', ' ', 'doing!', ' ', 'as', ' ', 'of', ' ', 'now?']


In [34]:
result = re.split(r'([.,!?] |\s)', text)
print(result)

['Hello', '. ', 'how', ' ', 'are', ' ', 'you', ' ', 'doing', '! ', 'as', ' ', 'of', ' ', 'now?']


In [35]:
result = [item for item in result if item.strip() ]
print(result)

['Hello', '. ', 'how', 'are', 'you', 'doing', '! ', 'as', 'of', 'now?']


In [36]:
text = "Hello, how' are you doing! as-- of? now?"

result = re.split(r'([,!:?_;()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)


['Hello', ',', 'how', "'", 'are', 'you', 'doing', '!', 'as', '--', 'of', '?', 'now', '?']


In [37]:
preprocessed = re.split(r'([,!:?_;()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:100])

['Harry', 'Potter', 'and', 'the', 'Sorcerer', "'", 's', 'Stone', 'CHAPTER', 'ONE', 'THE', 'BOY', 'WHO', 'LIVED', 'Mr.', 'and', 'Mrs.', 'Dursley', ',', 'of', 'number', 'four', ',', 'Privet', 'Drive', ',', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', ',', 'thank', 'you', 'very', 'much.', 'They', 'were', 'the', 'last', 'people', 'you', "'", 'd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', ',', 'because', 'they', 'just', 'didn', "'", 't', 'hold', 'with', 'such', 'nonsense.', 'Mr.', 'Dursley', 'was', 'the', 'director', 'of', 'a', 'firm', 'called', 'Grunnings', ',', 'which', 'made', 'drills.', 'He', 'was', 'a', 'big', ',', 'beefy', 'man', 'with', 'hardly', 'any', 'neck', ',', 'although', 'he', 'did', 'have', 'a', 'very']


In [38]:
print(len(preprocessed))

93107


# **Step 2 : Creating Tokens IDs**

> Add blockquote



In [39]:
all_words = sorted(set(preprocessed))
print(len(all_words))


9056


In [40]:
vocab = {token:integer for integer, token in enumerate(all_words)}


In [41]:
for i , item in enumerate(vocab.items()):
  print(item)
  if i >= 50:
    break

('!', 0)
('"', 1)
('"-', 2)
('"...', 3)
('"...for', 4)
('"0', 5)
('"A', 6)
('"AAAAAAAAAARGH', 7)
('"AAAARGH', 8)
('"ALL', 9)
('"ARE', 10)
('"Aaah', 11)
('"Aargh', 12)
('"Abbott', 13)
('"Abou', 14)
('"About', 15)
('"Absolutely', 16)
('"After', 17)
('"Ah', 18)
('"Aha', 19)
('"Ahem', 20)
('"Ahern', 21)
('"Alas', 22)
('"All', 23)
('"Always', 24)
('"Am', 25)
('"An', 26)
('"And', 27)
('"Another', 28)
('"Anyone', 29)
('"Anything', 30)
('"Anything.', 31)
('"Anyway', 32)
('"Are', 33)
('"Aren', 34)
('"As', 35)
('"Asked', 36)
('"At', 37)
('"Ate', 38)
('"B-b-but', 39)
('"B-but', 40)
('"Back', 41)
('"Bad', 42)
('"Barking', 43)
('"Be', 44)
('"Because', 45)
('"Best', 46)
('"Bet', 47)
('"Better', 48)
('"Bin', 49)
('"Blasted', 50)


In [57]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

Testing the tokenizer


In [58]:
tokenizer = SimpleTokenizerV1(vocab)

text = "THE BOY WHO LIVED"

ids = tokenizer.encode(text)
print(ids)


[1775, 695, 1899, 1273]


In [59]:
tokenizer.decode(ids)

'THE BOY WHO LIVED'

In [60]:
text = "Hello , do you like tea"
result = tokenizer.encode(text)
print(result)

KeyError: 'Hello'

# Adding Special Context Tokens

> Reason because of the key error getting due to the new words.



In [62]:
all_token = sorted(list(set(preprocessed)))
all_token.extend(["<|unk|>","<|endoftext|>"])

vocab = {token:integer for integer, token in enumerate(all_token)}

In [65]:
len(vocab.items())

9058

In [72]:
for i ,item in  enumerate(list(vocab.items())[-5:]):
  print(item)

('zoom', 9053)
('zoomed', 9054)
('zooming', 9055)
('<|unk|>', 9056)
('<|endoftext|>', 9057)


In [78]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        # added this to replace the new text or unkown text with this <|unk|>
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [82]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello do you like tea?"
text2 = "anything else you need while having tea "

text = " <|endoftext|> ".join((text1,text2))
print(text)

Hello do you like tea? <|endoftext|> anything else you need while having tea 


In [88]:
encoded = tokenizer.encode(text)
print(encoded)

[9056, 3597, 9031, 5399, 8030, 625, 9057, 2175, 3830, 9031, 5867, 8800, 4756, 8030]


In [89]:
tokenizer.decode(encoded)

'<|unk|> do you like tea? <|endoftext|> anything else you need while having tea'